Version 1.1.0

# The task

In this assignment you will need to implement features, based on nearest neighbours. 

KNN classifier (regressor) is a very powerful model, when the features are homogeneous and it is a very common practice to use KNN as first level model. In this homework we will extend KNN model and compute more features, based on nearest neighbors and their distances. 

You will need to implement a number of features, that were one of the key features, that leaded the instructors to prizes in [Otto](https://www.kaggle.com/c/otto-group-product-classification-challenge) and [Springleaf](https://www.kaggle.com/c/springleaf-marketing-response) competitions. Of course, the list of features you will need to implement can be extended, in fact in competitions the list was at least 3 times larger. So when solving a real competition do not hesitate to make up your own features.   

You can optionally implement multicore feature computation. Nearest neighbours are hard to compute so it is preferable to have a parallel version of the algorithm. In fact, it is really a cool skill to know how to use `multiprocessing`, `joblib` and etc. In this homework you will have a chance to see the benefits of parallel algorithm. 

# Check your versions

Some functions we use here are not present in old versions of the libraries, so make sure you have up-to-date software. 

In [1]:
import numpy as np
import pandas as pd 
import sklearn
import scipy.sparse 

for p in [np, pd, sklearn, scipy]:
    print (p.__name__, p.__version__)

numpy 1.13.1
pandas 0.20.3
sklearn 0.21.1
scipy 0.19.1


The versions should be not less than:

    numpy 1.13.1
    pandas 0.20.3
    sklearn 0.19.0
    scipy 0.19.1
   
**IMPORTANT!** The results with `scipy=1.0.0` will be different! Make sure you use _exactly_ version `0.19.1`.

# Load data

Learn features and labels. These features are actually OOF predictions of linear models.

In [3]:
!wget https://github.com/hse-aml/competitive-data-science/blob/master/readonly/KNN_features_data/X.npz
!wget https://github.com/hse-aml/competitive-data-science/blob/master/readonly/KNN_features_data/X_test.npz
!wget https://github.com/hse-aml/competitive-data-science/blob/master/readonly/KNN_features_data/Y.npy
!wget https://github.com/hse-aml/competitive-data-science/blob/master/readonly/KNN_features_data/Y_test.npy
!wget https://github.com/hse-aml/competitive-data-science/blob/master/readonly/KNN_features_data/knn_feats_test_first50.npy

--2019-05-29 07:50:56--  https://github.com/hse-aml/competitive-data-science/blob/master/readonly/KNN_features_data/X.npz
Resolving github.com (github.com)... 192.30.253.113
Connecting to github.com (github.com)|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘X.npz’

X.npz                   [ <=>                ]  62.38K  --.-KB/s    in 0.03s   

2019-05-29 07:50:56 (2.43 MB/s) - ‘X.npz’ saved [63881]

--2019-05-29 07:50:58--  https://github.com/hse-aml/competitive-data-science/blob/master/readonly/KNN_features_data/X_test.npz
Resolving github.com (github.com)... 192.30.253.112
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘X_test.npz’

X_test.npz              [ <=>                ]  62.40K  --.-KB/s    in 0.03s   

2019-05-29 07:50:58 (2.36 MB/s) - ‘X_test.npz’ saved [63895]

--2019-05-29 07:51:

In [0]:
import pickle

In [5]:
train_path = 'X.npz'
train_labels = 'Y.npy'

test_path = 'X_test.npz'
test_labels = 'Y_test.npy'

# Train data
X = scipy.sparse.load_npz(train_path)
Y = np.load(train_labels)

# Test data"
X_test = scipy.sparse.load_npz(test_path)
Y_test = np.load(test_labels)

# Out-of-fold features we loaded above were generated with n_splits=4 and skf seed 123
# So it is better to use seed 123 for generating KNN features as well 
skf_seed = 123
n_splits = 4

TypeError: ignored

Below you need to implement features, based on nearest neighbors.

In [0]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.neighbors import NearestNeighbors
from multiprocessing import Pool

import numpy as np


class NearestNeighborsFeats(BaseEstimator, ClassifierMixin):
    '''
        This class should implement KNN features extraction 
    '''
    def __init__(self, n_jobs, k_list, metric, n_classes=None, n_neighbors=None, eps=1e-6):
        self.n_jobs = n_jobs
        self.k_list = k_list
        self.metric = metric
        
        if n_neighbors is None:
            self.n_neighbors = max(k_list) 
        else:
            self.n_neighbors = n_neighbors
            
        self.eps = eps        
        self.n_classes_ = n_classes
    
    def fit(self, X, y):
        '''
            Set's up the train set and self.NN object
        '''
        # Create a NearestNeighbors (NN) object. We will use it in `predict` function 
        self.NN = NearestNeighbors(n_neighbors=max(self.k_list), 
                                      metric=self.metric, 
                                      n_jobs=1, 
                                      algorithm='brute' if self.metric=='cosine' else 'auto')
        self.NN.fit(X)
        
        # Store labels 
        self.y_train = y
        
        # Save how many classes we have
        self.n_classes = np.unique(y).shape[0] if self.n_classes_ is None else self.n_classes_
        
        
    def predict(self, X):       
        '''
            Produces KNN features for every object of a dataset X
        '''
        if self.n_jobs == 1:
            test_feats = []
            for i in range(X.shape[0]):
                test_feats.append(self.get_features_for_one(X[i:i+1]))
        else:
            '''
                 *Make it parallel*
                     Number of threads should be controlled by `self.n_jobs`  
                     
                     
                     You can use whatever you want to do it
                     For Python 3 the simplest option would be to use 
                     `multiprocessing.Pool` (but don't use `multiprocessing.dummy.Pool` here)
                     You may try use `joblib` but you will most likely encounter an error, 
                     that you will need to google up (and eventually it will work slowly)
                     
                     For Python 2 I also suggest using `multiprocessing.Pool` 
                     You will need to use a hint from this blog 
                     http://qingkaikong.blogspot.ru/2016/12/python-parallel-method-in-class.html
                     I could not get `joblib` working at all for this code 
                     (but in general `joblib` is very convenient)
                     
            '''
            
            # YOUR CODE GOES HERE
            # test_feats =  # YOUR CODE GOES HERE
            # YOUR CODE GOES HERE
            
            test_feats=[]
            pool=Pool(processes=self.n_jobs)
            
            batches=[]
            batch_size=(X.shape[0]//self.n_jobs)
            batch_size_total=0
            
            for i in range(self.n_jobs):
                batch=None
                if batch_size*(i+1)<=X.shape[0]:
                    batch=X[i*batch_size:batch_size*(i+1)]
                batch_size_total+=batch.shape[0]
                batches.append(batch)
                
                if i==self.n_jobs-1:
                    batch=X[(i+1)*batch_size:]
                    batch_size_total+=batch.shape[0]
                    batches.append(batch)
                
            print(X.shape[0])
            print(batch_size_total)
            assert batch_size_total==X.shape[0], 'Batch size not the same'
                  
            result=pool.map(self.get_features_for_batch,batches)
            for i in result:
                test_feats.extend(i)
                
            #assert False, 'You need to implement it for n_jobs > 1'
            
            
            
        return np.vstack(test_feats)
        
     
    def get_features_for_batch(self,batch):
        batch_feats=[]
        for i in range(batch.shape[0]):
            batch_feats.append(self.get_features_for_one(batch[i:i+1]))
        return batch_feats
        
    def get_features_for_one(self, x):
        '''
            Computes KNN features for a single object `x`
        '''

        NN_output = self.NN.kneighbors(x)
        
        # Vector of size `n_neighbors`
        # Stores indices of the neighbors
        neighs = NN_output[1][0]
        
        # Vector of size `n_neighbors`
        # Stores distances to corresponding neighbors
        neighs_dist = NN_output[0][0] 

        # Vector of size `n_neighbors`
        # Stores labels of corresponding neighbors
        neighs_y = self.y_train[neighs] 
        
        ## ========================================== ##
        ##              YOUR CODE BELOW
        ## ========================================== ##
        
        # We will accumulate the computed features here
        # Eventually it will be a list of lists or np.arrays
        # and we will use np.hstack to concatenate those
        return_list = [] 
        
        
        ''' 
            1. Fraction of objects of every class.
               It is basically a KNNСlassifiers predictions.

               Take a look at `np.bincount` function, it can be very helpful
               Note that the values should sum up to one
        '''
        for k in self.k_list:
            # YOUR CODE GOES HERE
            
            k_neighs=neighs[:k]
            k_neighs_y=self.y_train[k_neighs]
            feats=np.bincount(k_neighs_y,minlength=self.n_classes)
            feats_sum=np.sum(feats)
            feats=feats/feats_sum
            
            assert len(feats) == self.n_classes
            return_list += [feats]
        
        
        '''
            2. Same label streak: the largest number N, 
               such that N nearest neighbors have the same label.
               
               What can help you: `np.where`
        '''
        
        feats = 0# YOUR CODE GOES HERE
        N=0
        for k in range(1,max(self.k_list)+1):
            k_neighs=neighs[:k]
            k_neighs_y=self.y_train[k_neighs]
            k_same=np.where(k_neighs_y==k_neighs_y[0])[0]
            #print(k_same)
            if len(k_same)==k:
                N=k
        feats=[N]
        assert len(feats) == 1
        return_list += [feats]
        
        '''
            3. Minimum distance to objects of each class
               Find the first instance of a class and take its distance as features.
               
               If there are no neighboring objects of some classes, 
               Then set distance to that class to be 999.

               `np.where` might be helpful
        '''
        feats = np.ones(self.n_classes)*-1
        for c in range(self.n_classes):
            # YOUR CODE GOES HERE
            c_indices=np.where(neighs_y==c)[0]
            if len(c_indices)>0:
                c_neigh_dist=neighs_dist[c_indices]
                feats[c]=np.min(c_neigh_dist)
            else:
                feats[c]=999
                
        assert len(feats) == self.n_classes
        return_list += [feats]
        
        '''
            4. Minimum *normalized* distance to objects of each class
               As 3. but we normalize (divide) the distances
               by the distance to the closest neighbor.
               
               If there are no neighboring objects of some classes, 
               Then set distance to that class to be 999.
               
               Do not forget to add self.eps to denominator.
        '''
        feats = np.zeros(self.n_classes)#[]
        for c in range(self.n_classes):
            # YOUR CODE GOES HERE
            c_indices=np.where(neighs_y==c)[0]
            if len(c_indices)>0:
                c_neigh_dist=neighs_dist[c_indices]
                feats[c]=np.min(c_neigh_dist)/(np.min(neighs_dist)+self.eps)
            else:
                feats[c]=999
                
            
        assert len(feats) == self.n_classes
        return_list += [feats]
        
        '''
            5. 
               5.1 Distance to Kth neighbor
                   Think of this as of quantiles of a distribution
               5.2 Distance to Kth neighbor normalized by 
                   distance to the first neighbor
               
               feat_51, feat_52 are answers to 5.1. and 5.2.
               should be scalars
               
               Do not forget to add self.eps to denominator.
        '''
        for k in self.k_list:
            
            feat_51 = neighs_dist[k-1]# YOUR CODE GOES HERE
            feat_52 = feat_51/(neighs_dist[0]+self.eps)# YOUR CODE GOES HERE
            
            return_list += [[feat_51, feat_52]]
        
        '''
            6. Mean distance to neighbors of each class for each K from `k_list` 
                   For each class select the neighbors of that class among K nearest neighbors 
                   and compute the average distance to those objects
                   
                   If there are no objects of a certain class among K neighbors, set mean distance to 999
                   
               You can use `np.bincount` with appropriate weights
               Don't forget, that if you divide by something, 
               You need to add `self.eps` to denominator.
        '''
        
                
        
            # YOUR CODE GOES IN HERE
            
        for k in self.k_list:
            feats=np.zeros(self.n_classes)
            k_neighs_y=neighs_y[:k]
            for c in range(self.n_classes):
                c_indices=np.where(k_neighs_y==c)[0]
                if len(c_indices)>0:
                    feats[c]=np.sum(neighs_dist[c_indices])/(len(c_indices)+self.eps)
                else:
                    feats[c]=999
                        
            """   
            k_neighs=neighs[:k]
            k_neighs_y=neighs_y[:k]
            k_neighs_dist=neighs_dist[:k]
            
            feat=np.zeros(self.n_classes)
            c_count=np.bincount(k_neighs_y,minlength=self.n_classes)
            zero_indices=np.where(c_count==0)[0]
            feat[zero_indices]=999
            
            zipped=zip(k_neighs_y,k_neighs_dist)
            for c,w in zipped:
                feat[c]+=(w/(c_count[c]+self.eps))
            """
            
            assert len(feats) == self.n_classes
            return_list += [feats]
            
        
        # merge
        knn_feats = np.hstack(return_list)
        
        assert knn_feats.shape == (239,) or knn_feats.shape == (239, 1)
        return knn_feats

## Sanity check

To make sure you've implemented everything correctly we provide you the correct features for the first 50 objects.

In [0]:
# a list of K in KNN, starts with one 
k_list = [3, 8, 32]

# Load correct features
true_knn_feats_first50 = np.load('knn_feats_test_first50.npy')

# Create instance of our KNN feature extractor
NNF = NearestNeighborsFeats(n_jobs=1, k_list=k_list, metric='minkowski')

# Fit on train set
NNF.fit(X, Y)

# Get features for test
test_knn_feats = NNF.predict(X_test[:50])

#for i in zip(test_knn_feats[10],true_knn_feats_first50[10]):
#    print(i)
    
# This should be zero
print ('Deviation from ground thruth features: %f' % np.abs(test_knn_feats - true_knn_feats_first50).sum())

deviation =np.abs(test_knn_feats - true_knn_feats_first50).sum(0)
for m in np.where(deviation > 1e-3)[0]: 
    p = np.where(np.array([87, 88, 117, 146, 152, 239]) > m)[0][0]
    print ('There is a problem in feature %d, which is a part of section %d.' % (m, p + 1))

Deviation from ground thruth features: 0.000000


Now implement parallel computations and compute features for the train and test sets. 

## Get features for test

Now compute features for the whole test set.

In [0]:
for metric in ['minkowski', 'cosine']:
    print (metric)
    
    # Create instance of our KNN feature extractor
    NNF = NearestNeighborsFeats(n_jobs=4, k_list=k_list, metric=metric)
    
    # Fit on train set
    NNF.fit(X, Y)

    # Get features for test
    test_knn_feats = NNF.predict(X_test)
    
    # Dump the features to disk
    np.save('data/knn_feats_%s_test.npy' % metric , test_knn_feats)

minkowski
5586
5586
cosine
5586
5586


In [0]:
# Dump the features to disk
np.save('data/knn_feats_%s_test.npy' % metric , test_knn_feats)

## Get features for train

Compute features for train, using out-of-fold strategy.

In [0]:
# Differently from other homework we will not implement OOF predictions ourselves
# but use sklearn's `cross_val_predict`
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold

# We will use two metrics for KNN
for metric in ['minkowski', 'cosine']:
    print (metric)
    
    # Set up splitting scheme, use StratifiedKFold
    # use skf_seed and n_splits defined above with shuffle=True
    skf = StratifiedKFold(n_splits=n_splits,shuffle=True,random_state=skf_seed)# YOUR CODE GOES HERE
    
    # Create instance of our KNN feature extractor
    # n_jobs can be larger than the number of cores
    NNF = NearestNeighborsFeats(n_jobs=4, k_list=k_list, metric=metric)
    
    # Get KNN features using OOF use cross_val_predict with right parameters
    preds = cross_val_predict(NNF,X,Y,cv=skf.split(X,Y))# YOUR CODE GOES HERE
    
    # Save the features
    np.save('data/knn_feats_%s_train.npy' % metric, preds)

# Submit

If you made the above cells work, just run the following cell to produce a number to submit.

In [0]:
s = 0
for metric in ['minkowski', 'cosine']:
    knn_feats_train = np.load('data/knn_feats_%s_train.npy' % metric)
    knn_feats_test = np.load('data/knn_feats_%s_test.npy' % metric)
    
    s += knn_feats_train.mean() + knn_feats_test.mean()
    
answer = np.floor(s)
print (answer)

Submit!

In [0]:
from grader import Grader

grader.submit_tag('statistic', answer)

STUDENT_EMAIL = # EMAIL HERE
STUDENT_TOKEN = # TOKEN HERE
grader.status()

grader.submit(STUDENT_EMAIL, STUDENT_TOKEN)